In [17]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [18]:
from skforecast.ForecasterBaseline import ForecasterLastEquivalentDate
from skforecast.model_selection import backtesting_forecaster
import numpy as np
import pandas as pd
from skforecast import __version__
print(__version__)


0.10.1


In [19]:
# simulate a time series
n = 365
time = pd.date_range(start='2000-01-01', periods=n, freq='d')
y = np.arange(n)
df = pd.DataFrame({'time': time, 'y': y})
df = df.set_index('time')
df = df.asfreq('d')
df

,y
time,
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
...,...
2000-12-26,360
2000-12-27,361
2000-12-28,362


In [20]:
forecaster = ForecasterLastEquivalentDate(offset=7)
forecaster.fit(y=df['y'])
forecaster

ForecasterLastEquivalentDate 
Offset: 7 
Window size: 7 
Training range: [Timestamp('2000-01-01 00:00:00'), Timestamp('2000-12-30 00:00:00')] 
Training index type: DatetimeIndex 
Training index frequency: D 
Creation date: 2023-10-26 18:16:33 
Last fit date: 2023-10-26 18:16:33 
Skforecast version: 0.10.1 
Python version: 3.11.4 
Forecaster id: None 

In [21]:
forecaster.last_window

time
2000-12-24    358
2000-12-25    359
2000-12-26    360
2000-12-27    361
2000-12-28    362
2000-12-29    363
2000-12-30    364
Freq: D, Name: y, dtype: int32

In [22]:
forecaster.predict(steps=9)

2000-12-31    358
2001-01-01    359
2001-01-02    360
2001-01-03    361
2001-01-04    362
2001-01-05    363
2001-01-06    364
2001-01-07    358
2001-01-08    359
Freq: D, Name: pred, dtype: int32

In [23]:
metric, predictions = backtesting_forecaster(
    forecaster=forecaster,
    y=df['y'],
    initial_train_size=int(len(df) * 0.5),
    steps=9,
    metric='mean_absolute_error',
    verbose=True
)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 182
Number of observations used for backtesting: 183
    Number of folds: 21
    Number of steps per fold: 9
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 3 observations.

Fold: 0
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-01 00:00:00 -- 2000-07-09 00:00:00  (n=9)
Fold: 1
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-10 00:00:00 -- 2000-07-18 00:00:00  (n=9)
Fold: 2
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-19 00:00:00 -- 2000-07-27 00:00:00  (n=9)
Fold: 3
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)
    Validation: 2000-07-28 00:00:00 -- 2000-08-05 00:00:00  (n=9)
Fold: 4
    Training:   2000-01-01 00:00:00 -- 2000-06-30 00:00:00  (n=182)

  0%|          | 0/21 [00:00<?, ?it/s]

In [24]:
predictions

,pred
2000-07-01,175
2000-07-02,176
2000-07-03,177
2000-07-04,178
2000-07-05,179
...,...
2000-12-26,346
2000-12-27,347
2000-12-28,355
2000-12-29,356


In [25]:
import pandas as pd
from pandas.tseries import offsets
from pandas.tseries.offsets import DateOffset
ts = pd.Timestamp('2017-01-01 09:10:11')
ts + DateOffset(months=1)

Timestamp('2017-02-01 09:10:11')

In [26]:
offset = offsets.Day()
ts = pd.Timestamp('2017-01-01 09:10:11')
ts - offset

Timestamp('2016-12-31 09:10:11')